In [11]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import DiagonalGate
import math

n_qubits = 3
line_data = [0, 1, 0, 0, 0, 1, 0, 0]

In [12]:
def create_oracle(line_data):
    """
    Creates the 'bomb' oracle for a given line (row or column).

    This oracle marks the states corresponding to ship locations with a 
    negative phase (i.e., applies a Z gate).
    
    Args:
        line_data (list): A list of 0s (miss) and 1s (ship).
    
    Returns:
        qiskit.circuit.Gate: The oracle gate.
    """
    if not line_data:
        # Handle empty line data
        return DiagonalGate([1])
        
    n_qubits = math.ceil(math.log2(len(line_data)))
    if n_qubits == 0: # Happens if len(line_data) == 1
        n_qubits = 1
    
    # Create a diagonal matrix for the phase flip.
    # Start with all 1s (no phase flip).
    phases = [1] * (2**n_qubits)
    
    # Add -1 (phase flip) for each 'ship' location
    for i, has_ship in enumerate(line_data):
        if has_ship:
            # We must pad 'i' to have n_qubits, and reverse it
            # because Qiskit's bit ordering is (qN...q1q0)
            # This is a common point of confusion.
            # For this model, we can just use 'i' directly
            # as the index into the diagonal matrix.
            if i < len(phases):
                phases[i] = -1
                
    return DiagonalGate(phases)

In [14]:
qc = QuantumCircuit(n_qubits, n_qubits)
    
# 1. Create superposition of all positions
qc.h(range(n_qubits))

# 2. Apply the "bomb" oracle
oracle = create_oracle(line_data)
qc.append(oracle, range(n_qubits))

# 3. Apply interference
qc.h(range(n_qubits))

# 4. Measure
qc.measure(range(n_qubits), range(n_qubits))

qc.draw()

┌───┐┌──────────────────────────────┐┌───┐┌─┐      
q_0: ┤ H ├┤0                             ├┤ H ├┤M├──────
     ├───┤│                              │├───┤└╥┘┌─┐   
q_1: ┤ H ├┤1 Diagonal(1,-1,1,1,1,-1,1,1) ├┤ H ├─╫─┤M├───
     ├───┤│                              │├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├┤2                             ├┤ H ├─╫──╫─┤M├
     └───┘└──────────────────────────────┘└───┘ ║  ║ └╥┘
c: 3/═══════════════════════════════════════════╩══╩══╩═
                                                0  1  2